In [2]:
import pandas as pd

In [5]:
# Load the population data (from your previous work)
population_df = pd.read_csv("cc9_population_2024_percentages_for_map_corrected.csv")

In [10]:
# Load the care beds/personnel data
care_df = pd.read_csv("project_chart5_care_personnel_beds.csv", sep=";", encoding='latin-1')

In [12]:
print("Care data shape:", care_df.shape)
print("Population data shape:", population_df.shape)

Care data shape: (530, 4)
Population data shape: (476, 7)


In [13]:
# Ensure district codes are 5-digit strings with leading zeros in both datasets
care_df['district_code'] = care_df['district_code'].astype(str).str.zfill(5)
population_df['district_code'] = population_df['district_code'].astype(str).str.zfill(5)

In [23]:
# Convert numeric columns to float (in case they're strings)
care_df['available_beds'] = pd.to_numeric(care_df['available_beds'], errors='coerce')
care_df['personnel'] = pd.to_numeric(care_df['personnel'], errors='coerce')
population_df['total_population'] = pd.to_numeric(population_df['total_population'], errors='coerce')

In [24]:
# Merge: population_df as base, add care data to it
merged_df = population_df.merge(
    care_df[['district_code', 'available_beds', 'personnel']],
    on='district_code',
    how='left'
)

print(f"\nMerged data shape: {merged_df.shape}")
print(f"Districts with missing care data: {merged_df['available_beds'].isna().sum()}")


Merged data shape: (476, 9)
Districts with missing care data: 78


In [25]:
# Calculate beds per 1000 inhabitants
merged_df['beds_per_1000'] = (
    merged_df['available_beds'] / merged_df['total_population'] * 1000
).round(2)

In [26]:
# Calculate personnel per 1000 inhabitants
merged_df['personnel_per_1000'] = (
    merged_df['personnel'] / merged_df['total_population'] * 1000
).round(2)

In [27]:
# Create dataset for beds per 1000
beds_df = merged_df[[
    'district_code',
    'district_name',
    'total_population',
    'available_beds',
    'beds_per_1000'
]].copy()

In [28]:
beds_df

,district_code,district_name,total_population,available_beds,beds_per_1000
0,01001,"Flensburg, kreisfreie Stadt",96326.0,1092.0,11.34
1,01002,"Kiel, kreisfreie Stadt",252668.0,2297.0,9.09
2,01003,"Lübeck, kreisfreie Stadt",216889.0,3410.0,15.72
3,01004,"Neumünster, kreisfreie Stadt",79809.0,1197.0,15.00
4,01051,"Dithmarschen, Landkreis",133460.0,2118.0,15.87
...,...,...,...,...,...
471,16073,"Saalfeld-Rudolstadt, Landkreis",100105.0,1455.0,14.53
472,16074,Saale-Holzland-Kreis,83342.0,972.0,11.66
473,16075,Saale-Orla-Kreis,77341.0,873.0,11.29
474,16076,"Greiz, Landkreis",93707.0,1552.0,16.56


In [29]:
# Create dataset for personnel per 1000
personnel_df = merged_df[[
    'district_code',
    'district_name',
    'total_population',
    'personnel',
    'personnel_per_1000'
]].copy()

In [30]:
personnel_df

,district_code,district_name,total_population,personnel,personnel_per_1000
0,01001,"Flensburg, kreisfreie Stadt",96326.0,913.0,9.48
1,01002,"Kiel, kreisfreie Stadt",252668.0,1705.0,6.75
2,01003,"Lübeck, kreisfreie Stadt",216889.0,2836.0,13.08
3,01004,"Neumünster, kreisfreie Stadt",79809.0,882.0,11.05
4,01051,"Dithmarschen, Landkreis",133460.0,1770.0,13.26
...,...,...,...,...,...
471,16073,"Saalfeld-Rudolstadt, Landkreis",100105.0,1086.0,10.85
472,16074,Saale-Holzland-Kreis,83342.0,680.0,8.16
473,16075,Saale-Orla-Kreis,77341.0,617.0,7.98
474,16076,"Greiz, Landkreis",93707.0,1092.0,11.65


In [34]:
# Save to CSV files
beds_df.to_csv("project_chart5_care_beds_per_1000_inhabitants.csv", index=False, encoding='utf-8-sig')
personnel_df.to_csv("project_chart5_care_personnel_per_1000_inhabitants.csv", index=False, encoding='utf-8-sig')